![CSTR Diagram](../assets/Image_README.png)

# CSTR Simulator

A modular, GUI-integrated simulator for continuous stirred-tank reactors (CSTRs) with recycle loop, temperature optimization, and real industrial case studies.

This notebook was created as a collaborative project for the **EPFL** course _Practical Programming in Chemistry (**CHE-200**)_.

---

## Introduction

youpi

---

## Why Continuous Flow Stirred Reactors?

...